In [60]:
import os
import io
import dotenv
from pprint import pprint
import six
import pathlib
import json

import cv2

# Load env
dotenv.load_dotenv('../.env')

# Imports the Google Cloud client library
from google.cloud import vision
#from google.cloud.vision import types

# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

REQUIRED_ENTITIES = {'ORGANIZATION':'','PERSON':'','LOCATION':'','ADDRESS':'','PHONE_NUMBER':'',}

class BusinessCardReader:
    alpha = 0.3
    
    def __init__(self):
        pass
    
    def detect_text(self,path):
        """Detects text in the file."""
        from google.cloud import vision
        client = vision.ImageAnnotatorClient()
    
        with io.open(str(path), 'rb') as image_file:
            content = image_file.read()
    
        image = vision.types.Image(content=content)
    
        response = client.text_detection(image=image)
        texts = response.text_annotations
        
        #save result
        save_file_path = path.parent / 'output' / path.name
        
        #save overlayed image
        image = cv2.imread(str(path))
        overlay = image.copy()
        output = image.copy()
        for i,text in enumerate(texts):
            if(i == 0):
                continue
            
            cv2.rectangle(overlay,
                          (text.bounding_poly.vertices[0].x,text.bounding_poly.vertices[0].y),
                          (text.bounding_poly.vertices[2].x,text.bounding_poly.vertices[2].y),
                          (0,255,0),
                          -1)
        cv2.addWeighted(overlay, self.alpha, output, 1 - self.alpha,0, output)
        cv2.imwrite(str(save_file_path),output)
        
        #save raw text
        text_file_name = save_file_path.with_suffix('.txt')
        with io.open(str(text_file_name),'w') as file:
            print(response.text_annotations[0].description)
            file.write(response.text_annotations[0].description)
        
        return response
    
    def extract_entities(self,response,path):
        client = language.LanguageServiceClient()
        self.required_entities = {}
        self.required_entities.update(REQUIRED_ENTITIES)
        
        text = response.text_annotations[0].description
        
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        
        # Instantiates a plain text document.
        document = types.Document(
            content=text,
            type=enums.Document.Type.PLAIN_TEXT)
        
        # Detects entities in the document. You can also analyze HTML with:
        #   document.type == enums.Document.Type.HTML
        entities = client.analyze_entities(document).entities
        
        for entity in entities:
            entity_type = enums.Entity.Type(entity.type)
            if entity_type.name in self.required_entities:
                #print(entity.name)
                self.required_entities[entity_type.name] += entity.name
        pprint(self.required_entities)
        
        #save rabeling result
        save_file_path = path.parent / 'output' / path.name
        json_file_name = save_file_path.with_suffix('.json')
        #print(json_file_name)
        with io.open(str(json_file_name), 'w') as file:
            json.dump(self.required_entities,file)
        
    def save_result(self):
        pass
        
        
    def read_business_card(self,path):
        image_path_list = list(pathlib.Path(path).glob('*.jpg'))
        for image_path in image_path_list:
            response = self.detect_text(image_path)
            self.extract_entities(response, image_path)
            
        

test_url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/016.jpg'
image_path = '../images/001.jpg'
file_path = '../images/business_cards/Reference/'

business_card_reader = BusinessCardReader()
#business_card_reader.detect_text_uri(test_url)
#business_card_reader.extract_entities()
#business_card_reader.read_business_card_uri(test_url)

business_card_reader.read_business_card(file_path)


Institute for Computer Sciences, Social Informatics
and Telecommunications Engineering
ICST.ORG
Gabriella MAGYAR
Conference Coordinator
Begijnhoflaan 93a, B-9000 Gent, Belgium
phone: +32 9 3299 425
e-mail: gabriella.magyar@icst.org
skype: gabriella.magyar-icst
Gent Boston Hong Kong Sydney Alexandria
www

{'ADDRESS': 'Begijnhoflaan 93a, B-9000 Gent, Belgium',
 'LOCATION': 'GentBelgiumSydneyGent Boston Hong KongAlexandria',
 'ORGANIZATION': 'Institute for Computer Sciences, Social Informatics and '
                 'Telecommunications Engineering '
                 'ICST.ORGgabriella.magyar-icstgabriella.magyar@icst.org',
 'PERSON': 'Gabriella MAGYAR Conference',
 'PHONE_NUMBER': ''}
JIN SHO
陣匠
tel (650) 321
454 S California Ave Palo Alto CA 94306
jinshorestaurant.com

{'ADDRESS': '321\n454 S California Ave Palo Alto CA 94306',
 'LOCATION': '',
 'ORGANIZATION': 'California Ave Palo',
 'PERSON': '',
 'PHONE_NUMBER': ''}
RED
DOOR
Movie Rental
and Sales
2123 El Camino Re
Palo Alto, CA 943C


InvalidArgument: 400 The language tr is not supported for entity analysis.